In [22]:
import numpy as np

In [23]:
import sys
sys.path.append('./utils')

from data import Data
from models import Models
from tags import Tags
tags = Tags()

In [18]:
N_TAGS = 17

def threshold(pred, true):
    thres = [0] * N_TAGS
    for i in range(N_TAGS):
        print('Calc thresh for tag {}'.format(tags.idx_to_tag(i)))
        thres[i] = tag_threshold(pred[:,i], true[:,i])
    return thres

def amazon_score(tp, fp, fn):
    p = tp / (tp + fp) if tp + fp > 0 else 0
    r = tp / (tp + fn) if tp + fn > 0 else 0
    result = 5 * p * r / (4 * p + r) if 4 * p + r > 0 else 0
    return result

def tag_threshold(pred, true):
    pred = pred.reshape((len(pred)))
    true = true.reshape((len(true)))
    N_POS = sum(true)
    N_NEG = len(pred) - N_POS
    pair = list(zip(pred, true))
    pair.sort()
    tp, fp, tn, fn = N_POS, N_NEG, 0, 0
    tag_thres = 0
    max_score = amazon_score(tp, fp, fn)
    best_counts = ()
    for i in range(len(pred)):
        if pair[i][1] == 0:
            fp -= 1
            tn += 1
        else:
            tp -= 1
            fn += 1
        current_score = amazon_score(tp, fp, fn)
        if current_score > max_score:
            max_score = current_score
            tag_thres = pair[i][0]
            best_counts = (tp, fp, tn, fn)
    print('Best (tp, fp, tn, fn) is {}'.format(best_counts))
    return tag_thres

def calc_threshold(path):
    m = Models.load_resnet50(path)
    d = Data()
    X, y = d.get_fold(0)
    y_pred = m.predict(X)
    return threshold(y_pred, y)

In [24]:
def test():
    pred = np.array([[0.2], [0.6], [0.4], [0.8]])
    true = np.array([[0], [1], [0], [1]])
    thres = threshold(pred, true)
    return thres